In [1]:
import requests
import json
import pandas as pd
import urllib.parse
import folium
import geopandas as gpd
import csv
import time

In [2]:
# Yelp API key
API_KEY = '3c3W1ad7Fk346olp9ISvzsZ7syXj_JTvnwxHJ9YuGWrjTyJ_FhEz6HYIo_eyGOaK8RBmO9xhrd0bush2_Lu2pr5jiWPTXHCIeN8nDZi68U26pmC1dy2U7V-_vgeMZHYx'
# Latitude and longitude for Kadıköy
latitude = 40.9819
longitude = 29.0247

# Query for coffee shops
term = 'coffee'

# URL for Yelp API
url = 'https://api.yelp.com/v3/businesses/search'

# Headers with API key
headers = {
    'Authorization': 'Bearer %s' % API_KEY,
}


In [3]:
parsed_data = []
offset = 0
while True:
    # Parameters
    params = {
        'term': term,
        'latitude': latitude,
        'longitude': longitude,
        'limit': 50,
        'offset': offset,
    }
    
    # Send request and get response
    resp = requests.get(url=url, headers=headers, params=params)
    data = json.loads(resp.text)

    # Get the businesses data
    businesses = data.get('businesses', [])
    if not businesses:
        break

    # Parse the data
    for business in businesses:
        business_name = business['name']
        business_lat = business['coordinates']['latitude']
        business_lng = business['coordinates']['longitude']
        business_category = business['categories'][0]['title'] if business['categories'] else None
        business_rating = business['rating']
        business_review_count = business['review_count']
        parsed_data.append([business_name, business_lat, business_lng, business_category, business_rating, business_review_count])

    # Increase the offset by the number of results in this batch
    offset += len(businesses)

    # As to avoid hitting Yelp's rate limits, it's a good idea to wait some time before making the next request
    time.sleep(1)
    
# Convert to pandas DataFrame
import pandas as pd
dfcafes = pd.DataFrame(parsed_data, columns=['Name', 'Latitude', 'Longitude', 'Category', 'Rating', 'ReviewCount'])

# Drop any row with missing data
dfcafes = dfcafes.dropna()

In [4]:
print(type(data))

<class 'dict'>


In [5]:
neighborhood_list = ["Abbasağa, Beşiktaş",
                    "Akat, Beşiktaş",
                    "Arnavutköy, Beşiktaş",
                    "Balmumcu, Beşiktaş",
                    "Bebek, Beşiktaş",
                    "Cihannüma, Beşiktaş",
                    "Dikilitaş, Beşiktaş",
                    "Etiler, Beşiktaş",
                    "Gayrettepe, Beşiktaş",
                    "Konaklar, Beşiktaş",
                    "Kuruçeşme, Beşiktaş",
                    "Kültür, Beşiktaş",
                    "Levazım, Beşiktaş",
                    "Levent, Beşiktaş",
                    "Mecidiye, Beşiktaş",
                    "Muradiye, Beşiktaş",
                    "Nisbetiye, Beşiktaş",
                    "Ortaköy, Beşiktaş",
                    "Sinanpaşa, Beşiktaş",
                    "Türkali, Beşiktaş",
                    "Ulus, Beşiktaş",
                    "Vişnezade, Beşiktaş",
                    "Yıldız, Beşiktaş",
                    "Arap Cami, Beyoğlu",
                    "Asmalı Mescit, Beyoğlu",
                    "Bedrettin, Beyoğlu",
                    "Bereketzade, Beyoğlu",
                    "Bostan, Beyoğlu",
                    "Bülbül, Beyoğlu",
                    "Camiikebir, Beyoğlu",
                    "Cihangir, Beyoğlu",
                    "Çatma Mescit, Beyoğlu",
                    "Çukur, Beyoğlu",
                    "Emekyemez, Beyoğlu",
                    "Evliya Çelebi, Beyoğlu",
                    "Fetihtepe, Beyoğlu",
                    "Firuzağa, Beyoğlu",
                    "Gümüşsuyu, Beyoğlu",
                    "Hacıahmet, Beyoğlu",
                    "Hacımimi, Beyoğlu",
                    "Halıcıoğlu, Beyoğlu",
                    "Hüseyinağa, Beyoğlu",
                    "İstiklal, Beyoğlu",
                    "Kadı mehmet Efendi, Beyoğlu",
                    "Kalyoncu Kulluğu, Beyoğlu",
                    "Kamer Hatun, Beyoğlu",
                    "Kaptanpaşa, Beyoğlu",
                    "Katip mustafa Çelebi, Beyoğlu",
                    "Keçeci Piri, Beyoğlu",
                    "Kemankeş, Beyoğlu",
                    "Kılıçali Paşa, Beyoğlu",
                    "Kocatepe, Beyoğlu",
                    "Kulaksız, Beyoğlu",
                    "Kuloğlu, Beyoğlu",
                    "Küçük Piyale, Beyoğlu",
                    "Müeyyetzade, Beyoğlu",
                    "Ömer Avni, Beyoğlu",
                    "Örnektepe, Beyoğlu",
                    "Piripaşa Sokağı, Beyoğlu",
                    "Piyalepaşa, Beyoğlu",
                    "Pürtelaş Hasan Efendi, Beyoğlu",
                    "Sururi Mehmet Efendi, Beyoğlu",
                    "Sütlüce, Beyoğlu",
                    "Şahkulu, Beyoğlu",
                    "Şehit Muhtar, Beyoğlu",
                    "Tomtom, Beyoğlu",
                    "Yahya Kahya, Beyoğlu",
                    "Yenişehir, Beyoğlu",
                    "Aksaray, Fatih",
                    "Akşemsettin, Fatih",
                    "Alemdar, Fatih",
                    "Ali Kuşçu, Fatih",
                    "Atikali, Fatih",
                    "Ayvansaray, Fatih",
                    "Balabanağa, Fatih",
                    "Balat, Fatih",
                    "Beyazıt, Fatih",
                    "Binbirdirek, Fatih",
                    "Cankurtaran, Fatih",
                    "Cerrahpaşa, Fatih",
                    "Cibali, Fatih",
                    "Demirtaş, Fatih",
                    "Derviş Ali, Fatih",
                    "Emin Sinan, Fatih",
                    "Hacı Kadın, Fatih",
                    "Haseki Sultan, Fatih",
                    "Hırka-İ Şerif, Fatih",
                    "Hobyar, Fatih",
                    "Hoca Gıyasettin, Fatih",
                    "Hocapaşa, Fatih",
                    "İskenderpaşa, Fatih",
                    "Kalenderhane, Fatih",
                    "Karagümrük, Fatih",
                    "Katip Kasım, Fatih",
                    "Kemalpaşa, Fatih",
                    "Koca Mustafapaşa, Fatih",
                    "Küçük Ayasofya, Fatih",
                    "Mercan, Fatih",
                    "Mesihpaşa, Fatih",
                    "Mevlanakapı, Fatih",
                    "Mimar Hayrettin, Fatih",
                    "Mimar Kemalettin, Fatih",
                    "Molla Fenari, Fatih",
                    "Molla Gürani, Fatih",
                    "Molla Hüsrev, Fatih",
                    "Muhsine Hatun, Fatih",
                    "Nişanca, Fatih",
                    "Rüstempaşa, Fatih",
                    "Saraç İshak, Fatih",
                    "Seyyid Ömer, Fatih",
                    "Silivrikapı, Fatih",
                    "Sultan Ahmet, Fatih",
                    "Sururi, Fatih",
                    "Süleymaniye, Fatih",
                    "Sümbül Efendi, Fatih",
                    "Şehremini, Fatih",
                    "Şehsuvar Bey, Fatih",
                    "Tahtakale, Fatih",
                    "Taya Hatun, Fatih",
                    "Topkapı, Fatih",
                    "Yavuz Sinan, Fatih",
                    "Yavuz Sultan Selim, Fatih",
                    "Yedikule, Fatih",
                    "Zeyrek, Fatih",
                    "19 Mayıs, Kadıköy",
                    "Acıbadem, Kadıköy",
                    "Bostancı, Kadıköy",
                    "Caddebostan, Kadıköy",
                    "Caferağa, Kadıköy",
                    "Dumlupınar, Kadıköy",
                    "Eğitim, Kadıköy",
                    "Erenköy, Kadıköy",
                    "Fenerbahçe, Kadıköy",
                    "Feneryolu, Kadıköy",
                    "Fikirtepe, Kadıköy",
                    "Göztepe, Kadıköy",
                    "Hasanpaşa, Kadıköy",
                    "Koşuyolu, Kadıköy",
                    "Kozyatağı, Kadıköy",
                    "Merdivenköy, Kadıköy",
                    "Osmanağa, Kadıköy",
                    "Rasimpaşa, Kadıköy",
                    "Sahrayıcedit, Kadıköy",
                    "Suadiye, Kadıköy",
                    "Zühtüpaşa, Kadıköy",
                    "Acıbadem, Üsküdar",
                    "Ahmediye, Üsküdar",
                    "Altunizade, Üsküdar",
                    "Aziz Mahmut Hüdayi, Üsküdar",
                    "Bahçelievler, Üsküdar",
                    "Barbaros, Üsküdar",
                    "Beylerbeyi, Üsküdar",
                    "Bulgurlu, Üsküdar",
                    "Burhaniye, Üsküdar",
                    "Cumhuriyet, Üsküdar",
                    "Çengelköy, Üsküdar",
                    "Ferah, Üsküdar",
                    "Güzeltepe, Üsküdar",
                    "İcadiye, Üsküdar",
                    "Kandilli, Üsküdar",
                    "Kısıklı, Üsküdar",
                    "Kirazlıtepe, Üsküdar",
                    "Kuleli, Üsküdar",
                    "Kuzguncuk, Üsküdar",
                    "Küçük Çamlıca, Üsküdar",
                    "Küçüksu, Üsküdar",
                    "Küplüce, Üsküdar",
                    "Mehmet Akif Ersoy, Üsküdar",
                    "Mimar Sinan, Üsküdar",
                    "Muratreis, Üsküdar",
                    "Salacak, Üsküdar",
                    "Selami Ali, Üsküdar",
                    "Selimiye, Üsküdar",
                    "Sultantepe, Üsküdar",
                    "Ünalan, Üsküdar",
                    "Valide-İ Atik, Üsküdar",
                    "Yavuztürk, Üsküdar",
                    "Zeynep Kamil, Üsküdar"]

result_dict = {}


In [6]:
for neighborhood in neighborhood_list:
    request_text = urllib.parse.quote(neighborhood + " Istanbul")
    request_text = f"https://nominatim.openstreetmap.org/search?q={request_text}&polygon_geojson=1&format=json"
    response = requests.get(request_text)

    # decode API response
    response = response.json()

    # Check if response is not empty
    if response:
        # target neighborhood, assumes the first result in list is the correct one
        neighborhood_result = response[0]
        decoded_name = urllib.parse.unquote(neighborhood_result['display_name'])  # decode the name here
        feature = {
            "type": "Feature",
            "properties": {
                "name": decoded_name  # Use the decoded name here
            },
            "geometry": neighborhood_result['geojson']
        }
        result_dict[neighborhood] = feature
    else:
        print(f"No results for {neighborhood}")

# Put the features into a FeatureCollection
geojson_data = {
    "type": "FeatureCollection",
    "features": list(result_dict.values())
}


# Saving the modified geojson_data to a JSON file
with open('resultv2.json', 'w') as fp:
    json.dump(geojson_data, fp)


No results for Piripaşa Sokağı, Beyoğlu


In [7]:
districts = gpd.read_file('resultv2.json')
print(districts.head())


                                                name  \
0  Abbasağa Mahallesi, Beşiktaş, İstanbul, Marmar...   
1  Akat Mahallesi, Beşiktaş, İstanbul, Marmara Bö...   
2  Arnavutköy, Bebek Arnavutköy Caddesi, Arnavutk...   
3  Balmumcu Mahallesi, Beşiktaş, İstanbul, Marmar...   
4  Bebek Mahallesi, Beşiktaş, İstanbul, Marmara B...   

                                            geometry  
0  POLYGON ((29.00319 41.04803, 29.00321 41.04761...  
1  POLYGON ((29.01997 41.09076, 29.02036 41.08906...  
2                          POINT (29.04327 41.06718)  
3  POLYGON ((29.00983 41.05668, 29.01034 41.05653...  
4  POLYGON ((29.03265 41.07844, 29.03288 41.07817...  


In [8]:
population = pd.read_csv("C:\\Users\\nazli\\OneDrive\\Desktop\\DATA\\tüiklast2.csv")

In [9]:
merged = districts.set_index('name').join(population.set_index('name')).reset_index()

In [10]:
m = folium.Map(location=[41.0082, 28.9784], zoom_start=10)

choropleth = folium.Choropleth(
    geo_data=merged.__geo_interface__, 
    name='choropleth',
    data=merged,
    columns=['name', 'TOPLAM'],
    key_on='feature.properties.name',
    fill_color='OrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Population in Istanbul Districts'
)

choropleth.add_to(m)

m

In [11]:
# Add a marker for each coffee shop
for lat, lon, name in dfcafes[['Latitude', 'Longitude', 'Name']].values.tolist():
    folium.Marker(
        location=[lat, lon],
        popup=name,
        icon=folium.Icon(icon="cloud"),
    ).add_to(m)

# Save to HTML
m.save('ist_pop_cafe.html')

m


In [12]:
# Read the csv file
df = pd.read_csv('C:\\Users\\nazli\\OneDrive\\Desktop\\DATA\\AirbnbIstanbul.csv')

# Remove duplicates
df = df.drop_duplicates()

# Remove rows with 'n/a' in 'latitude' or 'longitude' columns
df = df[df['latitude'] != 'n/a']
df = df[df['longitude'] != 'n/a']


# 'Kadikoy' in 'neighbourhood' column
df = df[df['neighbourhood'].isin(['Kadikoy', 'Fatih', "Üsküdar", "Beyoğlu", 'Besiktas'])]
df = df[df['room_type'].isin(['Entire home/apt'])]
df = df[df['price'] < 1000]

#df = df[df['number_of_reviews'] != 0]
#df = df[df['reviews_per_month'] > 0.5]

# Save the cleaned data to a new csv file
df.to_csv('final_airbnb.csv', index=False)

df = df.dropna(subset=['neighbourhood'])

In [13]:
def get_semt(lat, lon):
    request_text = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json&zoom=18"
    response = requests.get(request_text)
    response = response.json()
    
    if "suburb" in response["address"]:
        semt = response["address"]["suburb"]
    else:
        semt = response["address"]["neighbourhood"]

    semt = semt.replace(" Mahallesi", "")
    ilce = response["address"]["town"]

    return semt, ilce, lat, lon

In [16]:
lat = 40.962856
lon = 29.061369


semt, ilce, latitude, longitude = get_semt(lat, lon)
print(f"Neighborhood: {semt}")
print(f"District: {ilce}")
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")


Neighborhood: Caddebostan
District: Kadıköy
Latitude: 40.962856
Longitude: 29.061369


In [ ]:
prices = {}
locations = {}

counter = 0  # To keep track of the number of entries processed

with open('final_airbnb.csv', 'r',  encoding='utf-8') as file:
    csv_file = csv.DictReader(file)

    for row in csv_file:
        lat, lon = float(row['latitude']), float(row['longitude'])
        price = int(row['price'])
        semt, ilce, lat, lon = get_semt(lat, lon)

        if semt in prices:
            prices[semt].append(price)
        else:
            prices[semt] = [price]
            locations[semt] = (lat, lon)
        
        # Increment the counter
        counter += 1

        # Print the dictionaries and break after processing 5 entries
        if counter == 400:
            #print("Prices so far:", prices)
            #print("Locations so far:", locations)
            break

In [ ]:

# Create the map
m = folium.Map(location=[41.0082, 28.9784], zoom_start=10)

choropleth = folium.Choropleth(
    geo_data=merged.__geo_interface__, 
    name='choropleth',
    data=merged,
    columns=['name', 'TOPLAM'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.2,
    legend_name='Population in Istanbul Districts'
)

choropleth.add_to(m)

# Add a marker for each coffee shop
for df in [dfcafes]:
    for lat, lon, name in df[['Latitude', 'Longitude', 'Name']].values.tolist():
        folium.Marker(
            location=[lat, lon],
            popup=name,
            icon=folium.Icon(icon="cloud"),
        ).add_to(m)

# Compute the average prices
average_prices = {semt: sum(price_list) / len(price_list) for semt, price_list in prices.items()}

# Add markers for each Airbnb location
for semt, (lat, lon) in locations.items():
    average_price = average_prices[semt]

    # Create a string that contains the information you want to display
    info = f"Semt: {semt}<br>Average Price: {average_price:.2f}"

    # Create a popup with the information
    popup = folium.Popup(info, max_width=250)

    # Add a circle marker for the location to the map
    folium.CircleMarker(
        location=[lat, lon],
        radius=average_price / 20,  # you might need to adjust the division factor to get appropriate circle sizes
        color="darkgrey",
        fill=True,
        fill_color="green",
        fill_opacity=0.6,
        popup=popup
    ).add_to(m)

m.save('istanbul_last.html')

m

In [ ]:
#unique neighborhoods in your population data
population_neighborhoods = set(population['name'].unique())

# features in your GeoJSON data
geojson_neighborhoods = set(feature['properties']['name'] for feature in geojson_data['features'])

#print(list(geojson_neighborhoods))

# check for differences
print("In population data but not in GeoJSON:", population_neighborhoods - geojson_neighborhoods)
print("In GeoJSON but not in population data:", geojson_neighborhoods - population_neighborhoods)

